In [1]:
import datetime as dt
import numpy as np 
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn.preprocessing as pp
from sklearn.cluster import KMeans, SpectralClustering, Birch
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

plt.style.use('fivethirtyeight')

# Data Cleaning

In [2]:
# Load data into pandas
customers = pd.read_csv("olist_customers_dataset.csv")
orders = pd.read_csv("olist_orders_dataset.csv")
payments = pd.read_csv("olist_order_payments_dataset.csv")
items = pd.read_csv("olist_order_items_dataset.csv")
reviews = pd.read_csv("olist_order_reviews_dataset.csv")
sellers = pd.read_csv("olist_sellers_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")
tproducts = pd.read_csv("product_category_name_translation.csv")
geolocation = pd.read_csv("olist_geolocation_dataset.csv")

In [3]:
# Datatype conversion for easier manipulation
df = customers.merge(orders, on = 'customer_id',how = 'left')
date_columns = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S')
df['order_purchase_year'] = df.order_purchase_timestamp.apply(lambda x: x.year)
df['order_purchase_month'] = df.order_purchase_timestamp.apply(lambda x: x.month)
df['order_purchase_dayofweek'] = df.order_purchase_timestamp.apply(lambda x: x.dayofweek)
df['order_purchase_hour'] = df.order_purchase_timestamp.apply(lambda x: x.hour)
df['order_purchase_day'] = df['order_purchase_dayofweek'].map({0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'})
df['order_purchase_mon'] = df.order_purchase_timestamp.apply(lambda x: x.month).map({1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'})

df['month_year'] = df['order_purchase_month'].astype(str).apply(lambda x: '0' + x if len(x) == 1 else x)
df['month_year'] = df['order_purchase_year'].astype(str) + '-' + df['month_year'].astype(str)
df['month_year2'] = df['order_purchase_mon'] + ' ' +  df['order_purchase_year'].astype(str)

In [4]:
df = df.merge(payments, on = 'order_id', how = 'left')
df = df.merge(items, on = 'order_id', how = 'left')
df = df.merge(products, on = 'product_id', how = 'left')
df = df.merge(tproducts, on = 'product_category_name', how = 'left')

In [5]:
df.dropna(subset=['payment_value'],inplace = True)
# dropping missing values
df.dropna(inplace=True)

# excluding incomplete 2012 data and displaying first 5 rows of master dataframe
df = df.query("month_year != '2016-12' and month_year != '2016-10'")

# RFM Segmentation

In [6]:
last_date = df['order_purchase_timestamp'].max() + dt.timedelta(days=1)

In [7]:
rfm2 = df.groupby('customer_unique_id').agg(
                 Recency = ('order_purchase_timestamp', lambda x: (last_date - x.max()).days),
                 Frequency = ('customer_id', lambda x: x.nunique()),
                 Monetary = ('price', 'sum'))

In [8]:
rfm2 = rfm2[rfm2['Frequency']<3]
rfm2 = rfm2[rfm2['Monetary']<1100]

In [11]:
rfm2cluster = rfm2[['Recency','Frequency','Monetary']]
rfm2_log = rfm2cluster[['Recency', 'Frequency', 'Monetary']].apply(np.log, axis = 1)

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(rfm2_log)
#Store it separately for clustering
rfm2_normalized = scaler.transform(rfm2_log)

# Cold Start

The clustering and segmentation we have done so far is only applicable to existing customers because they have the relevant *Recency, Frequency and Monetary* information. But what about new customers if they do not have the sufficient information? It leads us to the *cold start* problem where our system try to make inferences even if there are insufficient information.

In [13]:
# K-Means Clustering with 4 clusters
kmeans = KMeans(n_clusters=4, random_state = 1)
kmeans.fit(rfm2_normalized)
labels1 = kmeans.predict(rfm2_normalized)
rfm2['KCluster4'] = pd.Categorical(labels1)
RFMKCluster4 = rfm2.groupby('KCluster4').agg({'Recency':['mean'],'Frequency':['mean'],'Monetary':['mean','count']})

In [14]:
# Create a dataframe with Payments, Customers and Orders
PCO = pd.merge(orders,customers,on='customer_id')
PCO = pd.merge(PCO, payments, on ='order_id')

In [15]:
# Drop irrelevant columns
PCO.drop(columns = ['order_delivered_carrier_date', 'customer_zip_code_prefix', 'order_approved_at', 'order_status'],
         inplace = True)

# Remove any duplicated customer_unique_id
PCO.drop_duplicates(subset = ['customer_unique_id'], inplace = True)

In [16]:
# Reset the index of the dataframe with the rfm segmentation
rfm2.reset_index(inplace = True)

In [17]:
# This will give us the characteristic information of the different customers as well as their rfm cluster
PCO = pd.merge(PCO, rfm2[['customer_unique_id', 'KCluster4']], on = 'customer_unique_id')
PCO = pd.merge(PCO, items, on = 'order_id')
PCO.drop_duplicates(subset = 'customer_unique_id', inplace = True)

In [18]:
# Separate the dataframe into their clusters
cluster_0 = PCO[PCO['KCluster4'] == 0]
cluster_1 = PCO[PCO['KCluster4'] == 1]
cluster_2 = PCO[PCO['KCluster4'] == 2]
cluster_3 = PCO[PCO['KCluster4'] == 3]

In [19]:
# Converting delivery time to datetime datatype for easier manipulation
cluster_0['order_delivered_customer_date'] = pd.to_datetime(cluster_0['order_delivered_customer_date']
                                                            , format='%Y-%m-%d %H:%M:%S')
cluster_0['order_estimated_delivery_date'] = pd.to_datetime(cluster_0['order_estimated_delivery_date']
                                                            , format='%Y-%m-%d %H:%M:%S')
cluster_1['order_delivered_customer_date'] = pd.to_datetime(cluster_1['order_delivered_customer_date']
                                                            , format='%Y-%m-%d %H:%M:%S')
cluster_1['order_estimated_delivery_date'] = pd.to_datetime(cluster_1['order_estimated_delivery_date']
                                                            , format='%Y-%m-%d %H:%M:%S')
cluster_2['order_delivered_customer_date'] = pd.to_datetime(cluster_2['order_delivered_customer_date']
                                                            , format='%Y-%m-%d %H:%M:%S')
cluster_2['order_estimated_delivery_date'] = pd.to_datetime(cluster_2['order_estimated_delivery_date']
                                                            , format='%Y-%m-%d %H:%M:%S')
cluster_3['order_delivered_customer_date'] = pd.to_datetime(cluster_3['order_delivered_customer_date']
                                                            , format='%Y-%m-%d %H:%M:%S')
cluster_3['order_estimated_delivery_date'] = pd.to_datetime(cluster_3['order_estimated_delivery_date']
                                                            , format='%Y-%m-%d %H:%M:%S')

<ipython-input-19-631b0deb5e4f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_0['order_delivered_customer_date'] = pd.to_datetime(cluster_0['order_delivered_customer_date']
<ipython-input-19-631b0deb5e4f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_0['order_estimated_delivery_date'] = pd.to_datetime(cluster_0['order_estimated_delivery_date']
<ipython-input-19-631b0deb5e4f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

In [20]:
# Calculate the difference in delivery date and time
cluster_0['Difference_in_delivery_date'] = cluster_0['order_estimated_delivery_date'] - cluster_0['order_delivered_customer_date']
cluster_1['Difference_in_delivery_date'] = cluster_1['order_estimated_delivery_date'] - cluster_1['order_delivered_customer_date']
cluster_2['Difference_in_delivery_date'] = cluster_2['order_estimated_delivery_date'] - cluster_2['order_delivered_customer_date']
cluster_3['Difference_in_delivery_date'] = cluster_3['order_estimated_delivery_date'] - cluster_3['order_delivered_customer_date']


<ipython-input-20-c8a5235255eb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_0['Difference_in_delivery_date'] = cluster_0['order_estimated_delivery_date'] - cluster_0['order_delivered_customer_date']
<ipython-input-20-c8a5235255eb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_1['Difference_in_delivery_date'] = cluster_1['order_estimated_delivery_date'] - cluster_1['order_delivered_customer_date']
<ipython-input-20-c8a5235255eb>:4: SettingWithCopyWarning: 
A value is trying to be set 

In [21]:
cluster_0.reset_index(inplace = True)
cluster_1.reset_index(inplace = True)
cluster_2.reset_index(inplace = True)
cluster_3.reset_index(inplace = True)

In [22]:
# Changing the format of the datetime to just days
for i in range(len(cluster_0)):
    cluster_0['Difference_in_delivery_date'].loc[i] = cluster_0['Difference_in_delivery_date'].loc[i].days

for i in range(len(cluster_1)):
    cluster_1['Difference_in_delivery_date'].loc[i] = cluster_1['Difference_in_delivery_date'].loc[i].days
    
for i in range(len(cluster_2)):
    cluster_2['Difference_in_delivery_date'].loc[i] = cluster_2['Difference_in_delivery_date'].loc[i].days

for i in range(len(cluster_3)):
    cluster_3['Difference_in_delivery_date'].loc[i] = cluster_3['Difference_in_delivery_date'].loc[i].days

/apps/python/v3.8.5/lib/python3.8/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/apps/python/v3.8.5/lib/python3.8/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [23]:
# To drop irrelevant columns
clusters = [cluster_0, cluster_1, cluster_2, cluster_3]

for i in clusters:
    i.drop(columns = ['order_delivered_customer_date', 'order_estimated_delivery_date', 'customer_unique_id'], inplace = True)

/apps/python/v3.8.5/lib/python3.8/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Cold Start Solution:
***Assume that suppose a customer's characteristics matches the cluster's characteristics, we can put the customer into that cluster.***

1) Identify the characteristics of separate clusters.

2) Extract the same characteristics information of the new customers and do a comparison.

3) Assign the new customers to their clusters.

The characteristics we will look at are:
- Top 5 Locations
- Top 5 Products
- Top 5 States
- Payment Methods
- Difference in delivery date
- Installment plan
- Amount spent

In [24]:
def cluster_characteristics(cluster):
    df1 = pd.DataFrame()
    top5_customer_city = pd.DataFrame(cluster.customer_city.value_counts().head(5))
    top5_customer_city.reset_index(inplace = True)
    top5_customer_city.rename(columns = {'index': 'City'}, inplace = True)
    df1['Top 5 Countries'] = top5_customer_city.City
    
    top5_state = pd.DataFrame(cluster.customer_state.value_counts().head(5))
    top5_state.reset_index(inplace = True)
    top5_state.rename(columns = {'index': 'State'}, inplace = True)
    df1['Top 5 States'] = top5_state.State
    
    product = pd.merge(cluster['product_id'], products, on = 'product_id')
    product = pd.merge(product, tproducts, on = 'product_category_name')
    product_top5 = pd.DataFrame(product.product_category_name_english.value_counts().head(5))
    product_top5.reset_index(inplace = True)
    product_top5.rename(columns = {'index': 'Products'}, inplace = True)
    df1['Top 5 Products'] = product_top5.Products
    
    paymenttype = pd.DataFrame(cluster.payment_type.value_counts(normalize = True))
    paymenttype.reset_index(inplace = True)
    paymenttype.rename(columns = {'index': 'Payment_type'}, inplace = True)
    df1['Payment Type'] = paymenttype.Payment_type
    
    df1['Delivery Date Diff'] = cluster.Difference_in_delivery_date.mean().round(2)
    
    df1['Installment'] = cluster.payment_installments.mean().round(0)
    df1['Average Spending'] = '-'
    
    for i in [1,2,3,4]:
        df1["Delivery Date Diff"].loc[i] = '-'
        df1["Installment"].loc[i] = '-'
    df1.fillna('-', inplace = True)
    
    
    return df1

In [25]:
# Pass every cluster dataframe into the function
cluster0_characteristics = cluster_characteristics(cluster_0)
cluster1_characteristics = cluster_characteristics(cluster_1)
cluster2_characteristics = cluster_characteristics(cluster_2)
cluster3_characteristics = cluster_characteristics(cluster_3)

In [44]:
RFMKCluster4

Recency Frequency    Monetary       
                 mean      mean        mean  count
KCluster4                                         
0           40.521185         1  113.967644  15365
1          279.181139         1   44.343429  34918
2          220.283103         2  229.791263   2462
3          276.400696         1  210.462998  37949

In [35]:
cluster0_characteristics['Average Spending'].loc[0] = 114.0

In [36]:
cluster1_characteristics['Average Spending'].loc[0] = 44.3

In [37]:
cluster2_characteristics['Average Spending'].loc[0] = 230.0

In [38]:
cluster3_characteristics['Average Spending'].loc[0] = 210.4

In [39]:
cluster0_characteristics

,Top 5 Countries,Top 5 States,Top 5 Products,Payment Type,Delivery Date Diff,Installment,Average Spending
0,sao paulo,SP,health_beauty,credit_card,10.86,3.0,114.0
1,rio de janeiro,RJ,bed_bath_table,boleto,-,-,-
2,belo horizonte,MG,housewares,debit_card,-,-,-
3,brasilia,PR,watches_gifts,voucher,-,-,-
4,curitiba,RS,sports_leisure,-,-,-,-


In [40]:
cluster1_characteristics

,Top 5 Countries,Top 5 States,Top 5 Products,Payment Type,Delivery Date Diff,Installment,Average Spending
0,sao paulo,SP,bed_bath_table,credit_card,10.6,2.0,44.3
1,rio de janeiro,RJ,health_beauty,boleto,-,-,-
2,belo horizonte,MG,telephony,voucher,-,-,-
3,brasilia,RS,sports_leisure,debit_card,-,-,-
4,campinas,PR,furniture_decor,-,-,-,-


In [41]:
cluster2_characteristics

,Top 5 Countries,Top 5 States,Top 5 Products,Payment Type,Delivery Date Diff,Installment,Average Spending
0,sao paulo,SP,bed_bath_table,credit_card,11.23,3.0,230.0
1,rio de janeiro,RJ,furniture_decor,boleto,-,-,-
2,belo horizonte,MG,sports_leisure,voucher,-,-,-
3,brasilia,RS,health_beauty,debit_card,-,-,-
4,campinas,PR,computers_accessories,-,-,-,-


In [42]:
cluster3_characteristics

,Top 5 Countries,Top 5 States,Top 5 Products,Payment Type,Delivery Date Diff,Installment,Average Spending
0,sao paulo,SP,bed_bath_table,credit_card,10.86,4.0,210.4
1,rio de janeiro,RJ,health_beauty,boleto,-,-,-
2,belo horizonte,MG,sports_leisure,voucher,-,-,-
3,brasilia,RS,watches_gifts,debit_card,-,-,-
4,curitiba,PR,computers_accessories,-,-,-,-


# Conclusion

Unfortunately, the characteristics of each clusters are too similar to one another. So it is still difficult to assign new customers to their clusters. 
What we will need is more specific data like age, gender etc.